In [1]:
#importing necessary libraries

from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import numpy as np
import argparse
import cv2
import os
from sklearn.externals import joblib
from google.colab.patches import cv2_imshow

/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [2]:
# load our *serialized face detector model* from disk
print("[INFO] loading face detector model...")
net = cv2.dnn.readNet("deploy.prototxt", "res10_300x300_ssd_iter_140000.caffemodel")
#The structure of the neural network is stored in deploy.prototxt file
#The weights of the layers of the neural network are stored in res10_300x300_ssd_iter_140000.caffemodel file

# load the face mask detector model (*TRAINED PREVIOUSLY*) from disk
print("[INFO] loading face mask detector model...")
model = joblib.load('loaded_model_updated_dataset_3.pkl')


[INFO] loading face detector model...
[INFO] loading face mask detector model...


Using TensorFlow backend.
